# Transformation documentation

Load transformations file

In [127]:
import json
from IPython.display import JSON
with open('src/formulabar/transformations.json') as json_file:
    data = json.load(json_file)
    
data = data['transformations']

**Recommended**: Use jupyterlab table of contents extension for better navigation (@jupyterlab/toc)

# Introduction

Data transformations enable non-experts to wrangle and transform data without having to memorize syntax, thereby reducing the number of things one needs to memorize to use code to solve business problems (what we call the cognitive overhead). 

What users are doing right now is searching for many common data transformations. We improve this experience by reducing the friction of the process:

1. **Unified user flow:** The user does not need to switch back and forth between search/documentation and their coding environment
2. **Transformation curation:** The user does not need to navigate forums like Stackoverflow and spend time finding the best solution, we curate these for the user so that they can focus on solving business problems
3. **Less error-prone:** Sometimes, writing code as text is not the best UX: Python functions some-times take string as parameters, where only a limited set of strings is available. Allowing the user to type any text in a code editor can only lead to errors and slowness
4. **No re-inventing the wheel:** Millions of users around the world are googling and developing the same code. We want to enable people to focus on solving problems and reuse off-the-shelf components for common tasks

## Types of transformations

There are two type of transformations:



1. **Pandas functions:** This are the standard Pandas functions, where we provide a GUI that makes them easier to use
2. **Custom functions:**Custom functions are custom developed functions that are not part of the pandas library, but provide easy to use access to common functionality.

# A first simple transformation

## Understand the transformations.json file

## Showing a simple field

At the top-level of the file you have the function names

In [3]:
JSON(data)

<IPython.core.display.JSON object>

**Each function has two objects**
- **Form object**: This follows the structure defined in [reat-jsonschema-form](https://react-jsonschema-form.readthedocs.io/en/latest/) for auto-creating forms out of JSON schemas. Additionally, we have added a few custom fields for our implementation (labeled as custom), which are required for generating the code
- **UI schema:** This defines the UI aspects of the form we are generating, according to the [react-jsonschema-form UI reference](https://react-jsonschema-form.readthedocs.io/en/latest/api-reference/uiSchema/).

In [ ]:
JSON(data['read_csv'])

Let's look at the form object

In [4]:
JSON(data['read_csv']['form'])

<IPython.core.display.JSON object>

**We see the following keys:**
1. **properties**: Contains one element for each form field. Each field here will map to a parameter in a python function
2. **required**: Which of the parameters defined in properties is required (if not filled, returns a validation error)
3. **title**: The title of the function that is displayed in the UI. This should be easy to understand for a non-technical user. We follow a few conventions here:
    1. No underscores
    2. First letter is capital
3. **type**: Given JSONschema is used, we need to define the type of each element in the json tree. In this case, form is an object
4. **(custom) callerObject**: The object that can call this function 
    1. For example, read_csv is called from a pandas object, which we import as pd (a common way to import pandas, which we take as a convention). Thus we set the callerObject as pd
    2. [Learn more about how it is used in code generation](#understanding-code-generation)
5. **(custom) function**: function name in python (same as the outermost property)
6. **(custom) transformationType**: Shows the transformations when there is no data loaded. See reference for other options for transformationType

**Note that the keys labeled as custom are not part of react-jsonschema-form**

Let's see the properties object in more detail

In [ ]:
JSON(data['read_csv']['form']['properties'])

Fist we will look at the **new table name** field:

 **All of the transformations have a field called `New table name`**. This is a special field of type string (a simple text input) that will determine if the result of a transformation is save in a new variable (`df2 = df1.function(params)`) or not (`df1 = df1.function(params)`)

In [ ]:
JSON(data['read_csv']['form']['properties']['New table name'])

This is rendered as a simple text input

![alt text](img/transformation_guide_string_input.png)

We see that it has some placeholder text "Leave blank to modify selected table". This is defined in the UI schema

In [ ]:
JSON(data['read_csv']['uischema'],expanded=True)

 Each property in the form object can have UI schema attached to it. By default, the New table name has a UI placeholder that explains how it works.

## Adding more fields 

**properties:** If we review the [read_csv pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), we see it takes many parameters. Our approach here is to add the minimum numbers of parameters and only extend if we get user requests. In our case, we have decided to only keep three parameters: the filepath (a mandatory parameter), the separator and the decimal character. In the example below, you will see we set two additional fields
1. **type**: Next we need to define which types of parameters we have. In this case, we will use the string types, which just creates a string input
2. **title**: Creating a GUI gives us flexibility to override the pandas api and make it more user friendly, without worrying about creating long parameter names. Here the main goal is for the end-user to understand what that field does.
    1. Only define if it is not clear (e.g. by parameter is not easy to understand)
    2. Always lowercase (except new table name)
    3. Avoid underscores
    4. Keep it short, if too long use the description field
3. **description**: Provide additional information on what that function is doing 
4. **default**: Define a default value for the parameter

In [ ]:
JSON(data['read_csv']['form']['properties'], expanded=True)

**required**: **Here we mark that the filepath_or_buffer is a required parameter. This will trigger a validation error if not filled

In [ ]:
JSON(data['read_csv']['form']['required'], expanded=True)

<a id='understanding-code-generation' a>

## Understanding code generation 
When you run the transformation clicking submit, the code is mapped from a json object to a python function call:

**Input: Form data**


```json
{sep: ",", decimal: ".", filepath_or_buffer: "titanic.csv"}
```


**Output: Generated code**

```python
data = pd.read_csv(
    sep=",", 
    decimal=".", 
    filepath_or_buffer="titanic.csv")
```

**Formula:** result_object = caller_object dot function (parameters)


In this case, the ojbect is a dataframe
1. **result_object**: This comes from the “New table name” parameter. If empty, it overwrites the object that called it(`df1 = df1.function(params)`). If it is called from Pandas directly (i.e. not from a dataframe), it creates a dataframe called data as a default
2. **caller_object** : This is the caller-object defined before in the json schema
3. **function**: This is the function defined above in the json schema
4. **parameters**: These are the parameters passed from the form
    1. The way these are transformed is that it combines the key of the parameter with the value coming from the user input, using the equal sign that python functions use to specify parameters
    2. For example, <code>sep: ","</code>, becomes <code>sep="," </code>
    3. For readability, we format a parameter in each line
    4. Also notice that some parameters are implicit in python, meaning you don’t need to specify the <code>parametername=</code>. We nevertheless decide to do so to improve understanding of what the function is doing

## Test the transformation

Full code can be found [here](https://github.com/molinsp/eigendata/blob/0a23c3fee13846b90ff4af08c0a52d57696e9035/src/transformations.json#L160)



1. You can make changes to the JSON object and recompile the project (jlpm run build) to see changes in the UI. For example:
    1. Try adding a description to the filepath parameter
    2. Try adding an index_col parameter (see pandas reference)
    3. Try changing the defaults
2. Remember, the read_csv transformation is only visible when no data has been loaded

Adding a select dropdown in the sort function

## Preparing template for sort transformation
1. Based on the example above, we first prepare the boilerplate for the sort transformation
2. Notice how we are using a callerObject of type dataframe. This is because sort_values is a function available to Dataframe objects, so we define the callerObject as Dataframe (See code-generation to learn more) 

In [ ]:
boilerplate = {'sort_values':{'form': {}, 'uischema':{}}}
boilerplate['sort_values']['form'] = {
        'properties': {'New table name' :{'type': 'string'}},
        'title': 'Sort data',
        'description': 'Sort values based on a column',
        'type' : 'object',
        'required' : [],
        'callerObject': 'Dataframe',
        'function' : 'sort_values'
}

boilerplate['sort_values']['uischema'] = data['read_csv']['uischema']

JSON(boilerplate,expanded=True)

# Adding UI elements: dropdown

## Adding a simple dropdown

Here we will create a single select property:



1. enum: Define the values in the dropdown
2. enumNames: This is optional and allows you to give custom names in a dropdown. In this case, it is easier to read sorting type ascending than ascending=True
3. codegenstyle: This is not part of react-jsonschmea-form. This is a custom parameter that allows us to customize how we map the form response to the python code. As we saw in the example above, the default is pass adding string characters. In this case, that would result in somethinglike: ascending = “True”. Passing the codegenstyle=”variable” flag will remove the quotation marks form the user input, processing ascending=True instead.
4. default: This is the default value that will be populated in the UI. When there is both enum and enumNames, the default is taken from the enum.

In [ ]:
JSON(data['sort_values']['form']['properties']['ascending'], expanded=True)

![alt text](img/transformation_guide_select_field.png)

## Adding a dropdown with the names of the columns

Now sometimes, the dropdown values can not be defined in the jsonschema, because they depend on runtime information (e.g. which data frames have been loaded, what are the columns of a dataframe, etc.). In this case, we create a placeholder jsonschema and we auto-populate. Let’s see an example:



1. properties.by
    1. $ref: Here we are creating a reference to a definition object called columns. The idea is that if several parameters use the columns, we only have to define them once (learn more about definitions [here](https://react-jsonschema-form.readthedocs.io/en/latest/usage/definitions/)). 
    2. title: Here we may want to give a different name. We will always use the same standard reference to columns definition, and if needed change the name through the title
2. definitions: This holds the skeleton to populate the column dropdown. The empty enum will be populated with a list of all string names automatically

In [ ]:
JSON(data['sort_values']['form']['properties']['by'], expanded=True)

In [ ]:
JSON(data['sort_values']['form']['definitions'], expanded=True)

![alt text](img/transformation_guide_select_columns.png)

If we look at the code generation, we see that althrough we load the definitions of a parameter called column, the parameter we have rendered is called "by"
```python
data = data.sort_values(
    ascending=False, 
    by="Age")
```

# Adding a multi-select dropdown

The example below describes how to create a GUI for a multi-select using the column-names:



1. Add a reference to definitions/columns (notice the s at the end)
2. Add the columns definition code as-is

In [ ]:
JSON(data['get_dummies']['form']['properties']['columns'], expanded=True)

In [ ]:
JSON(data['get_dummies']['form']['definitions'], expanded=True)

![alt text](img/transformation_guide_multiselect_columns.png)

### Styling the multi-select differently
We can also use the UI-schema to style a multi-select differently

In [ ]:
JSON(data['drop']['form']['properties'],expanded=True)

In [ ]:
JSON(data['drop']['form']['definitions'], expanded=True)

In [ ]:
JSON(data['drop']['uischema']['columns'], expanded=True)

Here we have the same functionality but styled differently

![alt text](img/transformation_guide_multiselect_checkboxes.png)

### Understanding the code generation

**Input:** **Form data & selected dataframe**



1. Form data: 
```json
{columns: ["Parents/Children Aboard", "Siblings/Spouses Aboard"]}
```
2. Selected dataframe: data

<strong>Output: Generated code</strong>


```python
data = data.drop(
    columns=["Parents/Children Aboard","Siblings/Spouses Aboard"])
```



The input from the form is mapped to a generic formula in the following way:

Formula: dataframe = object dot function (parameters)
1. **dataframe**: Set to be the same than the selected datafreme given there is no input in New table name
2. **object** : Here, the callerObject DataFrame is replaced by the selected dataframe that is passed
3. **function**: This is the function defined above in the json schema

# Adding complex fields to the form

This example describes how to create UIs that are rendered as dictionaries in python
```python
property={'input1_field1' : 'input1_field2', 'input2_field1' : 'input2_field2'}`
```


The way this is implemented is using an array that has two sub-fields. The sub-fields are implementes nested property, i.e. a property inside another property:
1. In the example below, you see a property named dtype
2. The type is set to array
3. We defined an **items** element of type object
4. We define another properties element inside items (here is where the nesting takes place)

In [ ]:
JSON(data['astype']['form']['properties'], expanded=True)

# Transformations that return series

A series transformation is a transformation that return a [series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html)

Thus, the code we generate is of the type
```python
dataframe['Series name'] = dataframe['Series name'] dot function (parameters)
```

When we write a series transformation, we need to:

1. Specify an additional column parameter
2. Set the caller object to DataFrame[Series]
2. **Set codegenstyle: ignore, given that the column is not a real parameter of the function that nees to be rendered!**
3. The `new table property` is converted into a `new column property`, allowing us either to replace the column or define a new column

In [2]:
JSON(data['round'], expanded=True)

<IPython.core.display.JSON object>

# Transformations that return variables

A transformation can also return a variable that is not a dataframe/series. We currently call "variable" a few of the core python data types:
- Int/Float
- String
- List

In [3]:
JSON(data['mean']['form'], expanded=True)

<IPython.core.display.JSON object>

# Transformations with conditional logic

Sometimes, functions have more than one way of being called. This is usually hard for a user to understand when they read a documentation. How we have implemented this patter is with conditional fields in jsonschema form.

First, we define a **mode** property: 
1. With an enum that lists all the options. Make sure to **add mode to the list of required properties**. Given it is a dropdown with default value, it is not strictly required but we add it for consistency reasons
2. If the mode is not an explicit parameter in the code, make sure to set codegenstyle to ignore

**Example with mode as an actual parameter in the code**

In [7]:
JSON(data['bin_column']['form']['properties'], expanded=True)

<IPython.core.display.JSON object>

**Example with mode as an actual parameter in the code**

In [5]:
JSON(data['divide']['form']['properties'], expanded=True)

<IPython.core.display.JSON object>

To get the conditional behavior, we define dependencies for the mode property:
1. Define a **oneOf** object, with one element per mode. Each element inside the oneOf needs to be a valid jsonschema form
2. Create a property called mode, with an enum holding only one option. This means that this element of oneOf defines the case where the mode is equal to this value
3. Define the rest of the properties, including new column name/new table name, etc.

**(!) The elements in oneOf need to be in the same order than the enum element of the mode property**

In [7]:
JSON(data['divide']['form']['dependencies'], expanded=True)

<IPython.core.display.JSON object>

Lastly, make sure to define the return type as series

In [8]:
JSON(data['divide']['form'], expanded=False)

<IPython.core.display.JSON object>

# Transformations without return type

Some transformations do not return an object and just write to a notebook. In this case, you can set returnType to "none"

In [5]:
JSON(data['to_csv']['form'], expanded=False)

<IPython.core.display.JSON object>

If the return type is not specified, it will be determined by the 

# Importing libraries

# Importing snippets

# Selection as parameter & pure functions

Sometimes, a python transformation is not of the type `object = object.function(parameters)` but rather `object = function(parameters)`.

In this cases, many times the first positional parameter is the "main" elment of the transformation and the rest of the parameters are configurations. We can treat these cases in the same way from a UI perspective by setting the flag `selectionAsParameter`. 

![alt text](img/transformation_guide_selection_as_parameter.png)

In the case above, the code will be generated in the following way
`pd.get_dummies(data, columns='...')`

This way we can give users one single way of interacting and avoid the confusion generated by the fact that there are two different syntax patterns

In [4]:
JSON(data['get_dummies']['form'], expanded=False)

<IPython.core.display.JSON object>

The above is even more interesting when we apply it to pure functions, which have no caller object

In [12]:
JSON(data['euclidean_distance']['form'], expanded=False)

<IPython.core.display.JSON object>

# Trasnformation reference

Examples of react-jsonschema-form [here](https://rjsf-team.github.io/react-jsonschema-form/)

## Form
#### JSONschema form
1. **title**: Make the API more user-friendly
    1. No underscores, e.g. from `sort_by →  sort by`
    2. Use full words instead of abbreviations: `pat →  pattern`
    3. Help better understand parameters that take more than one type, e.g. sheet name can be
        1. `sheet name` if using the name
        2. `sheet number` if using an id
    4. If the API is very confusing, e.g. `na_sentinel → NaN value`
2. **description**: Add a description for each transformation.  
    1. Finalize sentences with dots
3. **required**
    9. Use property object name and not title
4. **definitions**
    
#### Eigendata custom properties
1. **callerObject**: DataFrame is replaced by the selected dataframe. Series by the seleciton in the column parameter
    1. `pd`
    2. `DataFrame` (for dataframe functions)
    3. `DataFrame.fdt` (for fastdata functions)
    4. `DataFrame[Series]` (for series functions)
    5. `DataFrame[Series].str`
    6. …
2. **function**: function to be called
3. **transformationType**: If not defined, we assume it is a function
    1. `dataLoading` Will be shown when there is no data
    2. `property` Will be called withot parameter parenthesis (e.g. dataframe.shape)
3. **returnType**: Type of object that will be returned. We are using the "new variable name" or "new column name" to infer it. Otherwise it is set to dataframe.
    1. `none` the result will not be assigned to anything (e.g. dataframe.to_csv(...))
    2. `series` the result will be a series
    3. `dataframe` the result will be dataframe
4. **selectionAsParameter**: See examples above
  
    
### Form properties
#### JSONschema form
1. **title**:
    1. Only define if it is not clear (e.g. by parameter is not easy to understand)
    2. Always lowercase (except new table name)
    3. Use title if parameter name has undercores
    4. Keep it short, if too long use the description field
2. **description**
    5. Always add a description
3. **type**: Use only two types from jsonschema-form
    6. `string`: For single select and string input
    7. `number`: For number inputs (e.g. like round example)
    8. `array`: For multi-select or array input
4. **\$ref**: Used to define single-select columns, multi-select columns and single-select data frames. No need to define type if $ref is used. Options are:
    9. `#/definitions/column`
    10. `#/definitions/columns`
    11. `#/definitions/dataframes`
5. **default**: In case there is enum and enumNames, the default should be the enum value and not the enumNames 

#### Eigendata custom properties
1. **ED: codegenstyle**: Not specified: Pass the input with string `property='input'`
    1. `variable` Pass the input without strings  `property=input`
    2. `ignore` Used for series transformations
    3. `seriesColumn` Used when the series is not passed as a string but rather as an object with the syntax dataFrame['series']

### uiSchema

# Tests

## Sanity tests

In [60]:
n_transformations = len(data)

In [77]:
def check_percentage_form(obj, search_property, get_missing=False):
    missing = []
    
    n_with_property = 0
    n_total = 0

    for i in data:
        n_total += 1
        if search_property in data[i]['form']:
            n_with_property += 1
        else:
            missing.append(i)
        

    percentage_with_property = n_with_property/n_total * 100.0
    print("Percentage of elements with {}: {}".format(search_property, percentage_with_property))
    
    if get_missing==True:
        return missing

In [78]:
check_percentage_form(data, 'title')

Percentage of elements with title: 100.0


In [79]:
check_percentage_form(data, 'description')

Percentage of elements with description: 100.0


In [80]:
check_percentage_form(data, 'required', get_missing=True)

Percentage of elements with required: 93.18181818181817


['describe', 'info', 'extract_number_from_string']

In [81]:
check_percentage_form(data, 'properties', get_missing=True)

Percentage of elements with properties: 95.45454545454545


['describe', 'info']

In [99]:
def check_percentage_properties(obj, search_property, get_missing=False):
    missing = []
    
    n_with_property = 0
    n_total = 0

    for i in data:
        if 'properties' in  data[i]['form']:
            
            properties = data[i]['form']['properties']
            for prop in properties:
                n_total += 1
                if search_property in properties[prop]:
                    n_with_property += 1
                else:
                    missing.append({'transformation': i, 'property': prop})

    percentage_with_property = n_with_property/n_total * 100.0
    print("Percentage of elements with {}: {}".format(search_property, percentage_with_property))
    
    if get_missing==True:
        return missing

In [101]:
check_percentage_properties(data, 'title', get_missing=False)

Percentage of elements with title: 49.137931034482754


In [102]:
check_percentage_properties(data, 'codegenstyle', get_missing=False)

Percentage of elements with codegenstyle: 38.793103448275865


## Consistency tests

### Properties do not have underscores

In [2]:
def check_no_property_name_with_underscore(transformations, verbose=False):
    n_with_property = 0
    n_total = 0
    missing = []

    # iterate over transformations
    for i in data:
        if 'properties' in  data[i]['form']:
            
            properties = data[i]['form']['properties']
            # look at the properties of all transformations
            for prop in properties:
                # check the transformations that have an underscore in the title
                if '_' in prop:
                    n_total += 1
                    if 'title' in properties[prop]:
                        if verbose: print('Element: ' + prop + ' , Title: ' + properties[prop]['title'])
                        n_with_property += 1
                    else:
                        print('No title defined in transformation ' + i + ' for property ' + prop)
    
    percentage_with_property = n_with_property/n_total * 100.0  
    print(missing)
    return percentage_with_property

In [5]:
check_no_property_name_with_underscore(data)

[]


100.0

In [3]:
assert check_no_property_name_with_underscore(data) == 100.0

[]


### Properties do not have different titles

Check that properties with the same name do not have different titles

#### Calculate number of replacements with query paths

In [66]:
import jmespath

In [67]:
standard_properties_with_title = 0
for i in jmespath.search('*.form.properties.*.title', data):
    #print(len(i))
    standard_properties_with_title += len(i)
standard_properties_with_title

47

In [68]:
dependency_properties_with_title = 0
for i in jmespath.search('*.form.dependencies.mode.oneOf[*].properties.*.title', data):
    for mode in i:
        dependency_properties_with_title += len(mode)

dependency_properties_with_title

16

In [69]:
standard_properties_with_title + dependency_properties_with_title

63

#### Analyze properties with different titles

In [128]:
def check_properties_with_title(properties_and_titles, properties):
    # look at the properties of all transformations
    for prop in properties:
        # check if we have seen it before
        if prop in properties_and_titles:
            # if it has a title
            # and we don't have that title for that property
            if 'title' in properties[prop] and (properties[prop]['title'] not in properties_and_titles[prop]['titles']):
                properties_and_titles[prop]['titles'].append(properties[prop]['title'])
                properties_and_titles[prop]['counter'] += 1
                
            elif 'title' in properties[prop]:
                 properties_and_titles[prop]['counter'] += 1
        else:
            # check if it has a title that is not the parameter name
            if 'title' in properties[prop]:
                properties_and_titles[prop] = {
                    'titles': [properties[prop]['title']],
                    'counter': 1
                } 
            # we also need to check if we have renamed some properties with titles
            # and left others without titles
            else:
                properties_and_titles[prop] = {
                    'titles': [prop],
                    'counter': 1
                } 

In [129]:
def get_all_properties_with_title(transformations, verbose=False):
    properties_and_titles = {}
    
    # iterate over transformations
    for i in data:
        if 'properties' in  data[i]['form']:
            
            properties = data[i]['form']['properties']
            
            check_properties_with_title(properties_and_titles, properties)
                        
        # look at the dependencies
        if 'dependencies' in  data[i]['form']:
            for mode in data[i]['form']['dependencies']['mode']['oneOf']:
                check_properties_with_title(properties_and_titles, mode['properties'])
                
    return properties_and_titles

In [130]:
def check_properties_with_different_titles(properties, verbose=False):
    duplicated = {}
    for key in properties:
        if len(properties[key]['titles']) > 1:
            duplicated[key] = properties[key]
    
    return duplicated

In [131]:
check_properties_with_different_titles(get_all_properties_with_title(data))

{'sheet_name': {'titles': ['sheet number', 'sheet name'], 'counter': 2},
 'columns': {'titles': ['columns', 'Columns to rename'], 'counter': 2},
 'other': {'titles': ['other', 'value', 'column'], 'counter': 11}}

#### Check renamings that are not just removed whitespaces

In [132]:
def check_properties_with_title_no_whitespaces(properties_and_titles, properties):
    # look at the properties of all transformations
    for prop in properties:
        # check if we have seen it before
        if prop in properties_and_titles:
            # if it has a title
            # and we don't have that title for that property
            if 'title' in properties[prop] \
                and (properties[prop]['title'] not in properties_and_titles[prop]['titles'])\
                and (properties[prop]['title'] != prop.replace('_',' ')):
                
                properties_and_titles[prop]['titles'].append(properties[prop]['title'])
                properties_and_titles[prop]['counter'] += 1
                
            elif 'title' in properties[prop] and (properties[prop]['title'] != prop.replace('_',' ')):
                 properties_and_titles[prop]['counter'] += 1
        else:
            # check if it has a title that is not the parameter name
            if 'title' in properties[prop] and (properties[prop]['title'] != prop.replace('_',' ')):
                properties_and_titles[prop] = {
                    'titles': [properties[prop]['title']],
                    'counter': 1
                } 

In [133]:
def get_all_properties_with_different_title(transformations, verbose=False):
    properties_and_titles = {}
    
    # iterate over transformations
    for i in data:
        if 'properties' in  data[i]['form']:
            
            properties = data[i]['form']['properties']
            
            check_properties_with_title_no_whitespaces(properties_and_titles, properties)
                        
        # look at the dependencies
        if 'dependencies' in  data[i]['form']:
            for mode in data[i]['form']['dependencies']['mode']['oneOf']:
                check_properties_with_title_no_whitespaces(properties_and_titles, mode['properties'])
                
    return properties_and_titles

In [134]:
get_all_properties_with_different_title(data)

{'filepath_or_buffer': {'titles': ['file path'], 'counter': 1},
 'sep': {'titles': ['separator'], 'counter': 1},
 'sheet_name': {'titles': ['sheet number'], 'counter': 1},
 'path_or_buf': {'titles': ['file path'], 'counter': 1},
 'aggfunc': {'titles': ['aggregation'], 'counter': 2},
 'dummy_na': {'titles': ['NaN column'], 'counter': 1},
 'left_on': {'titles': ['left column'], 'counter': 1},
 'right_on': {'titles': ['right column'], 'counter': 1},
 'by': {'titles': ['column'], 'counter': 1},
 'regex': {'titles': ['regular expression'], 'counter': 3},
 'columns': {'titles': ['Columns to rename'], 'counter': 1},
 'dtype': {'titles': ['Columns to reassign'], 'counter': 1},
 'pat': {'titles': ['pattern'], 'counter': 2},
 'case': {'titles': ['case sensitive'], 'counter': 2},
 'sub': {'titles': ['substring'], 'counter': 1},
 'repl': {'titles': ['replacement text'], 'counter': 1},
 'n': {'titles': ['Number of replacements'], 'counter': 1},
 'other': {'titles': ['value', 'column'], 'counter': 1

# Export to csv

In [26]:
transformationlist_csv = 'transformation \n'
for i in data:
    transformationlist_csv += str(i) + '\n'
    
transformationlist_csv = transformationlist_csv[:-1]

In [27]:
f = open("transformation_list.csv", "w")
f.write(transformationlist_csv)
f.close()